# Meta

In [3]:
# Autoreload modules without having to restart the notebook kernel.
% load_ext autoreload
% autoreload 2


# Plotting code from Georg's notebook.
import matplotlib.pyplot as plt

% matplotlib inline
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 32}
plt.rc('font', **font)

UsageError: Line magic function `%` not found.


# Imports

In [8]:
# Make the source directory available on all machines.
import sys

# This must be executed before importing any files from the 'lib' module which exists in the src directory.
sys.path.insert(1, "..\\")

import networkx as nx

# Personal libraries
import lib.graph
import lib.files

# Constants

In [5]:
ICP55 = '4932.YER078C'
THRESHOLD = 900

### ICP55 cc 900 inv
ICP55's largest connected component with edge threshold 900 and inviable proteins included.

In [4]:
n0 = lib.graph.read_STRING()
n0 = lib.graph.remove_edges_below_threshold(n0, THRESHOLD)
n0 = lib.graph.remove_nodes_with_degree_lte(n0, 0)
n0 = lib.graph.get_largest_cc_with_node(n0, ICP55)
lib.graph.write_weighted_edgelist(n0, lib.files.make_filepath_to_networks('icp55-cc-900-inv.txt'))

### ICP55 cc 900 noInv
ICP55's largest connected component with edge threshold 900 and invialbe proteins exlcuded.

In [7]:
# Read in inviable_proteins as a list.
inviable_proteins = lib.graph.read_inviable_proteins(as_graph=False)

# Preprocess the network
n1 = lib.graph.read_STRING()
n1 = lib.graph.remove_edges_below_threshold(n1, THRESHOLD)
n1 = lib.graph.remove_nodes_from_list(n1, inviable_proteins)
n1 = lib.graph.remove_nodes_with_degree_lte(n1, 0)
n1 = lib.graph.get_largest_cc_with_node(n1, ICP55)

# Write to file
lib.graph.write_weighted_edgelist(n1, lib.files.make_filepath_to_networks('icp55-cc-900-noInv.txt'))